# Create FAPESP Awards from Virtual Library

Creates FAPESP awards from the Biblioteca Virtual (Virtual Library) data. ~113K research grants.

**Prerequisites:**
- Run `scripts/local/fapesp_to_s3.py` to download and upload the data first.

**Data source:** https://bv.fapesp.br/en/  
**CSV sources:**
- Ongoing: https://media.fapesp.br/bv/uploads/gera_csv_assincrono/csv/auxilios_em_andamento.csv
- Completed: https://media.fapesp.br/bv/uploads/gera_csv_assincrono/csv/auxilios_concluidos.csv

**S3 location:** `s3a://openalex-ingest/awards/fapesp/fapesp_projects.parquet`

**FAPESP funder:**
- funder_id: 4320320997
- display_name: "Fundação de Amparo à Pesquisa do Estado de São Paulo"
- ROR: https://ror.org/02ddkpn78
- DOI: 10.13039/501100001807

## Step 1: Create Staging Table from S3

In [ ]:
%sql
-- Create the staging table from S3 parquet
CREATE OR REPLACE TABLE openalex.awards.fapesp_raw
USING delta
AS
SELECT
    *,
    current_timestamp() as databricks_ingested_at
FROM parquet.`s3a://openalex-ingest/awards/fapesp/fapesp_projects.parquet`;

In [ ]:
%sql
-- Check row count (should be ~113K)
SELECT COUNT(*) as total_projects FROM openalex.awards.fapesp_raw;

In [ ]:
%sql
-- Sample the raw data
SELECT * FROM openalex.awards.fapesp_raw LIMIT 5;

In [ ]:
%sql
-- Verify column names before transformation
DESCRIBE openalex.awards.fapesp_raw;

## Step 2: Create FAPESP Awards Table

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.fapesp_awards
USING delta
AS
WITH
-- Get FAPESP funder from OpenAlex by explicit funder_id
fapesp_funder AS (
    SELECT
        funder_id,
        display_name,
        ror_id,
        doi
    FROM openalex.common.funder
    WHERE funder_id = 4320320997  -- FAPESP
),

awards_transformed AS (
    SELECT
        -- Generate unique ID using xxhash64 of funder_id:process_number
        abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.process_number)))) % 9000000000 as id,

        -- Display name = project title (prefer English, fallback to Portuguese)
        COALESCE(NULLIF(TRIM(g.title_en), ''), g.title_pt) as display_name,

        -- Description = abstract (prefer English, fallback to Portuguese)
        COALESCE(NULLIF(TRIM(g.abstract_en), ''), g.abstract_pt) as description,

        -- Funder info
        f.funder_id,
        g.process_number as funder_award_id,

        -- FAPESP data doesn't include funding amounts
        CAST(NULL AS DOUBLE) as amount,
        'BRL' as currency,

        -- Funder struct
        struct(
            CONCAT('https://openalex.org/F', f.funder_id) as id,
            f.display_name,
            f.ror_id,
            f.doi
        ) as funder,

        -- Funding type - map FAPESP support modalities to types
        CASE
            -- Fellowships and scholarships
            WHEN LOWER(g.support_modality) LIKE '%bolsa%' THEN 'fellowship'
            WHEN LOWER(g.support_modality) LIKE '%jovens pesquisadores%' THEN 'fellowship'
            WHEN LOWER(g.support_modality) LIKE '%young investigators%' THEN 'fellowship'
            WHEN LOWER(g.support_modality) LIKE '%pesquisador visitante%' THEN 'fellowship'
            WHEN LOWER(g.support_modality) LIKE '%visiting researcher%' THEN 'fellowship'
            -- Infrastructure
            WHEN LOWER(g.support_modality) LIKE '%infraestrutura%' THEN 'infrastructure'
            WHEN LOWER(g.support_modality) LIKE '%equipamentos%' THEN 'infrastructure'
            -- Conferences and meetings
            WHEN LOWER(g.support_modality) LIKE '%reunião%' THEN 'conference'
            WHEN LOWER(g.support_modality) LIKE '%organização%' THEN 'conference'
            -- Publications
            WHEN LOWER(g.support_modality) LIKE '%publicações%' THEN 'publication'
            -- Innovation/industry
            WHEN LOWER(g.support_modality) LIKE '%pipe%' THEN 'innovation'
            WHEN LOWER(g.support_modality) LIKE '%pite%' THEN 'innovation'
            WHEN LOWER(g.support_modality) LIKE '%inovativa%' THEN 'innovation'
            -- Thematic projects
            WHEN LOWER(g.support_modality) LIKE '%temático%' THEN 'research'
            WHEN LOWER(g.support_modality) LIKE '%thematic%' THEN 'research'
            -- Default: regular research grant
            ELSE 'grant'
        END as funding_type,

        -- Funder scheme = support modality
        g.support_modality as funder_scheme,

        -- Provenance
        'fapesp_bv' as provenance,

        -- Dates (stored as strings in YYYY-MM-DD format)
        TRY_TO_DATE(g.start_date, 'yyyy-MM-dd') as start_date,
        TRY_TO_DATE(g.end_date, 'yyyy-MM-dd') as end_date,
        TRY_CAST(YEAR(TRY_TO_DATE(g.start_date, 'yyyy-MM-dd')) AS INT) as start_year,
        TRY_CAST(YEAR(TRY_TO_DATE(g.end_date, 'yyyy-MM-dd')) AS INT) as end_year,

        -- Lead investigator
        CASE
            WHEN g.pi_family_name IS NOT NULL THEN
                struct(
                    g.pi_given_name as given_name,
                    g.pi_family_name as family_name,
                    CAST(NULL AS STRING) as orcid,  -- FAPESP data doesn't include ORCID
                    CAST(NULL AS DATE) as role_start,
                    struct(
                        g.institution as name,
                        'Brazil' as country,
                        CAST(NULL AS ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>) as ids
                    ) as affiliation
                )
            ELSE NULL
        END as lead_investigator,

        -- Co-lead and other investigators (not easily parsed from FAPESP data)
        CAST(NULL AS STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >) as co_lead_investigator,

        CAST(NULL AS ARRAY<STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >>) as investigators,

        -- Landing page URL (FAPESP Virtual Library search)
        CONCAT('https://bv.fapesp.br/en/auxilios/', REPLACE(g.process_number, '/', '-'), '/') as landing_page_url,

        -- No DOI for FAPESP grants typically
        CAST(NULL AS STRING) as doi,

        -- Works API URL
        concat('https://api.openalex.org/works?filter=awards.id:G', abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.process_number)))) % 9000000000) as works_api_url,

        -- Timestamps
        current_timestamp() as created_date,
        current_timestamp() as updated_date

    FROM openalex.awards.fapesp_raw g
    CROSS JOIN fapesp_funder f
    WHERE g.process_number IS NOT NULL
      AND TRIM(g.process_number) != ''
)

SELECT * FROM awards_transformed;

## Step 3: Insert into openalex_awards_raw

In [ ]:
%sql
-- Remove previous data for this source before inserting fresh data
DELETE FROM openalex.awards.openalex_awards_raw
WHERE provenance = 'fapesp_bv' AND priority = 19;

-- Insert into openalex_awards_raw with priority
INSERT INTO openalex.awards.openalex_awards_raw
SELECT
    id,
    display_name,
    description,
    funder_id,
    funder_award_id,
    amount,
    currency,
    funder,
    funding_type,
    funder_scheme,
    provenance,
    start_date,
    end_date,
    start_year,
    end_year,
    lead_investigator,
    co_lead_investigator,
    investigators,
    landing_page_url,
    doi,
    works_api_url,
    created_date,
    updated_date,
    19 as priority  -- FAPESP priority
FROM openalex.awards.fapesp_awards;

## Verification Queries

In [ ]:
%sql
-- Check row count (should be ~113K)
SELECT COUNT(*) as total_fapesp_awards FROM openalex.awards.fapesp_awards;

In [ ]:
%sql
-- Sample the data
SELECT 
    id,
    display_name,
    funder_award_id,
    funder_scheme,
    funding_type,
    start_date,
    end_date,
    lead_investigator
FROM openalex.awards.fapesp_awards 
LIMIT 10;

In [ ]:
%sql
-- Check funder distribution (should all be FAPESP)
SELECT funder.display_name, COUNT(*) as cnt
FROM openalex.awards.fapesp_awards
GROUP BY funder.display_name
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funding_type distribution
SELECT funding_type, COUNT(*) as cnt
FROM openalex.awards.fapesp_awards
GROUP BY funding_type
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funder_scheme distribution (top 20 support modalities)
SELECT funder_scheme, COUNT(*) as cnt
FROM openalex.awards.fapesp_awards
WHERE funder_scheme IS NOT NULL
GROUP BY funder_scheme
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check data completeness
SELECT
    COUNT(*) as total,
    COUNT(display_name) as has_title,
    COUNT(description) as has_abstract,
    COUNT(start_date) as has_start_date,
    COUNT(end_date) as has_end_date,
    COUNT(lead_investigator) as has_pi,
    ROUND(COUNT(lead_investigator) * 100.0 / COUNT(*), 1) as pct_with_pi,
    ROUND(COUNT(description) * 100.0 / COUNT(*), 1) as pct_with_abstract
FROM openalex.awards.fapesp_awards;

In [ ]:
%sql
-- Check year distribution
SELECT start_year, COUNT(*) as cnt
FROM openalex.awards.fapesp_awards
WHERE start_year IS NOT NULL
GROUP BY start_year
ORDER BY start_year DESC
LIMIT 20;

In [ ]:
%sql
-- Check lead institutions
SELECT 
    lead_investigator.affiliation.name as institution,
    COUNT(*) as cnt
FROM openalex.awards.fapesp_awards
WHERE lead_investigator.affiliation.name IS NOT NULL
GROUP BY lead_investigator.affiliation.name
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check knowledge areas
SELECT 
    major_knowledge_area,
    COUNT(*) as cnt
FROM openalex.awards.fapesp_raw
WHERE major_knowledge_area IS NOT NULL
GROUP BY major_knowledge_area
ORDER BY cnt DESC;